In [2]:
import geopandas as gpd 
import geemap
import rasterio
import ee 

In [3]:
ee.Authenticate()
ee.Initialize()

# Manejo de archivos vectoriales

In [14]:
edom= gpd.read_file(r'C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex.shp')

In [16]:
# Simplificar las geometrías con una tolerancia deseada. Ajusta este valor según sea necesario.
# La tolerancia estará en las mismas unidades que el sistema de coordenadas del shapefile.
gdf_simplified = edom.simplify(tolerance=1.0)

# Guardar el shapefile simplificado
#gdf_simplified.to_file(r'C:\Users\egonz\Documents\Curso\Variables\Edomex\prueba.shp')

In [17]:
import os

# Crea un mapa interactivo
#Map = geemap.Map()

# Ruta al archivo shapefile
#shp_file_path = r'C:\Users\egonz\Documents\Curso\Variables\Edomex\prueba.shp'

# Comprueba si el archivo shapefile existe
if not os.path.exists(shp_file_path):
    raise FileNotFoundError("El archivo shapefile no se encontró.")

# Convierte el shapefile a un Earth Engine FeatureCollection
ee_object = geemap.shp_to_ee(shp_file_path)

# Añade la capa al mapa
#Map.addLayer(ee_object, {}, 'Nombre de la capa')

# Muestra el mapa
#Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Manejo de archivos raster

# Temperatura

In [6]:
# Recortar la imagen para evitar trabajar con bits innecesarios

mask= r"C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex.shp"
temp= r"C:\Users\egonz\Documents\Curso\Variables\Insumos\Temp.tif"
out= r"C:\Users\egonz\Documents\Curso\Variables\TempClip.tif"

#tempClip= geemap.clip_image(temp,mask,out)

In [7]:
import numpy as np

## Funcional no función

In [34]:
""" import rasterio
#import numpy as np

# Abre el archivo raster original
with rasterio.open(r'C:\Users\egonz\Documents\Curso\Variables\TempClip.tif') as src:
    raster_data = src.read(1)  # Lee la primera banda
    profile = src.profile

# Define tus reglas de reclasificación, por ejemplo:
# Si el valor de píxel está entre 0 y 100, se reclasifica a 1
# Si el valor de píxel está entre 101 y 200, se reclasifica a 2
# Si el valor de píxel es mayor que 200, se reclasifica a 3
reclassified_data = np.where((raster_data > 0) & (raster_data <= 100), 1, raster_data)
reclassified_data = np.where((raster_data > 100) & (raster_data <= 200), 2, reclassified_data)
reclassified_data = np.where(raster_data > 200, 3, reclassified_data)

# Guarda el raster reclasificado
with rasterio.open(r'C:\Users\egonz\Documents\Curso\Variables\TempReclass.tif', 'w', **profile) as dst:
    dst.write(reclassified_data, 1)
 """

## Funcion recursiva

In [38]:
def reclasifica_raster(nombre_archivo, intervalos_valores, ruta_salida):
    with rio.open(nombre_archivo) as src:
        raster = src.read(1)
        raster_reclasificado = np.copy(raster)
        
        # Iterar sobre cada intervalo y su nuevo valor
        for intervalo in intervalos_valores:
            # Verificar que la tupla contenga exactamente dos elementos
            if len(intervalo) != 2:
                raise ValueError("Cada tupla debe tener exactamente dos elementos (intervalo, nuevo_valor)")
            rango, nuevo_valor = intervalo

            # Aplicar el nuevo valor a los píxeles dentro del intervalo
            raster_reclasificado = np.where(
                (raster > rango[0]) & (raster <= rango[1]),
                nuevo_valor,
                raster_reclasificado
            )

        # Escribir el raster reclasificado en la ruta de salida
        srcprof = src.profile.copy()
        srcprof.update(dtype=raster_reclasificado.dtype)
        with rio.open(ruta_salida, 'w', **srcprof) as dst:
            dst.write(raster_reclasificado, 1)

In [41]:
nombre_archivo= r'C:\Users\egonz\Documents\Curso\Variables\TempClip.tif'
lista_intervalos_valores= [
    ((-np.inf,10),np.nan),
    ((10,10),10),
    ((10,12),20),
    ((12,15),70),
    ((15, 22), 100),
    ((22,28),85),
    ((28,32),70),
    ((32,40),20),
    ((40,40),10),
    ((40,np.inf),np.nan)
]
ruta_salida= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\TempReclas.tif'

In [44]:
#reclasifica_raster(nombre_archivo=nombre_archivo, intervalos_valores=lista_intervalos_valores, ruta_salida=ruta_salida)

# Precipitación

In [45]:
precip= r'C:\Users\egonz\Documents\Curso\Variables\Insumos\PrecipAnual.tif'
reclass= [
    ((-np.inf,400),10),
    ((400,500),20),
    ((500,600),75),
    ((600,800),100),
    ((800,1200),95),
    ((1200,1400),75),
    ((1400,1600),20),
    ((1600,np.inf),10)
]
out= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\PrecipReclass.tif'

In [46]:
#reclasifica_raster(nombre_archivo=precip, intervalos_valores=reclass, ruta_salida=out)

In [48]:
pr= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\PrecipReclass.tif'
out= r'C:\Users\egonz\Documents\Curso\Variables\PrecipRec_Clip.tif'

In [49]:
geemap.clip_image(pr,mask,out)

# Altura

In [50]:
dem = ee.Image('USGS/SRTMGL1_003')
visAlt = {
    'min': 0,
    'max': 4000,
    'palette': ['00A8E2', '3DA832', 'FFDB5C', 'FF7E5C', 'DB1C00']
}
out= r'C:\Users\egonz\Documents\Curso\Variables\Insumos\Alturas.tif'


In [53]:
# Función para obtener el bounding box de un archivo .tif
def obtener_bbox(ruta_archivo):
    with rasterio.open(ruta_archivo) as dataset:
        bbox = dataset.bounds
        return bbox

# Función para convertir el bounding box a una geometría de Earth Engine
def bbox_a_ee_geometry(bbox):
    # Crea una lista con las coordenadas del bounding box
    coords = [[bbox.left, bbox.bottom], [bbox.left, bbox.top],
              [bbox.right, bbox.top], [bbox.right, bbox.bottom],
              [bbox.left, bbox.bottom]] # Cerrar el polígono
    # Crea un polígono con las coordenadas en Earth Engine
    poligono = ee.Geometry.Polygon(coords)
    return poligono

ruta_archivo= r'C:\Users\egonz\Documents\Curso\Variables\Insumos\PrecipAnual.tif'

# Obtén el bounding box del archivo .tif
bbox = obtener_bbox(ruta_archivo)

# Convierte el bounding box a una geometría de Earth Engine
poligono_ee = bbox_a_ee_geometry(bbox)

In [58]:
#AltClip= dem.clip(poligono_ee)